In [25]:
import json
import pandas as pd
import requests

In [26]:
follows = pd.read_csv('data/follow-link.csv')

In [27]:
usernames = pd.unique(follows[['source', 'target']].values.ravel()).tolist()

In [28]:
with open("keys3.txt") as f:
    lines = f.readlines()
    client_key = lines[0].strip()
    client_secret = lines[1].strip()

In [29]:
def get_access_token(client_key, client_secret):
    # Endpoint URL
    endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"

    # Request headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    # Request body parameters
    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
    }

    # Make the POST request
    response = requests.post(endpoint_url, headers=headers, data=data)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and print the response JSON
        response_json = response.json()
        return response_json['access_token']
    else:
        # If the request was not successful, print the error response JSON
        
        return response.json()

In [30]:
def get_liked_info(username, access_token,fields ="id, username" ):
    query_params = {"fields": fields}
    query_body = {"username": username}
    headers = {"Authorization": f"Bearer {access_token}"}
    
    endpoint = "https://open.tiktokapis.com/v2/research/user/liked_videos/"
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)

    if response.status_code == 200:
        # Parse and extract information from the response
        return response.json().get("data", {})
    else:
        
        return {response.text}

In [31]:
access_token = get_access_token(client_key, client_secret)

KeyError: 'access_token'

In [ ]:
liked_videos = []
for username in usernames[2989:3990]:
    liked_videos.append(get_liked_info(username, access_token))

In [ ]:
combined = {}
for i, username in enumerate(usernames[2989:3990]):
    combined[username] = liked_videos[i]

In [ ]:
rows = []

for key, value in combined.items():
    if value and 'user_liked_videos' in value:  # Ensure key has data
        for video in value['user_liked_videos']:
            rows.append({
                'username': key,  # Dictionary key
                'id': video['id'],  # Video ID
                'creator': video['username']  # Video username
            })

# Create DataFrame
likes_df = pd.DataFrame(rows)

# Display DataFrame
print(likes_df)

              username                   id           creator
0       severe_central  7472968292395748650  backpackwithzach
1       severe_central  7472774014398729514        careysface
2       severe_central  7471700025685429509         frost5841
3       severe_central  7459919709115387142     trendformusic
4       severe_central  7472859293486730539        evanspicks
..                 ...                  ...               ...
775  inthistogetherusa  7197191438461914414   citizensclimate
776  inthistogetherusa  7200874279859146027   citizensclimate
777  inthistogetherusa  7215275734456159530    connect.effect
778  inthistogetherusa  7242939119079361835            mnrris
779  inthistogetherusa  7427269576586530094     bryanna.bakes

[780 rows x 3 columns]


In [ ]:
likes_df.to_csv("liked_videos.csv", mode="a", index=False, header=False)